앨런 튜링의 튜링머신과 마찬가지로 람다계산법은 
수학자 다비드 힐베르트가 제시한 수리명제자동판별문제(Entscheidungsproblem)의
해결 불가능함을 보이기 위해 알론조 처치가 최소한의 문법으로 고안한
이론적 소품으로써 탄생했다. 이후 람다계산법은 프로그래밍 언어 이론의 연구에
활발히 활용되고 있으며 특히 함수형 프로그래밍언어 설계의 기초가 되고 있다.

이론적으로는 순수한 람다계산법만으로도 조건문, 자연수 등을 포함한 모든 계산이 표현 가능지만
실제 컴퓨터에서 그런 방식으로 계산한다면 비효율적일 것이다.
람다계산법을 기초로 하여 컴퓨터 하드웨어에서 제공하는 정수 연산 등
여러 가지 다양한 기능을 덧붙여 나간 언어들이 바로 함수형 언어다.
또한, 최근에는 함수형 언어로 시작하지 않은 많은 프로그래밍언어들도
람다계산법의 문법이나 개념을 도입하고 있기도 하므로 실무적 프로그래밍
능력 향상을 위해서도 람다계산법을 바탕으로 하는 언어에 대한 기본적 이해가 필요하다.

이 장에서는 지금까지 살펴본 덧셈식과 람다식 그리고 조건식까지 포함하는
언어에 대한 인자 먼저 값계산(call-by-value evaluation)을 하는
인터프리터를 하스켈로 작성해 본다.

\newpage

\begin{align*}
x &\;\in\; \texttt{Nm} \qquad\qquad n \;\in\; \mathbb{Z} \\
e &\;\in \texttt{Expr}
    ~::=~ x ~\mid~ \lambda x.e ~\mid~ e_1~e_2 
    ~\mid~ n ~\mid~ e_1 +\, e_2
    ~\mid~ \textbf{if}~e~\textbf{then}~e_1~\textbf{else}~e_0
\\[-3em]
\end{align*}
\begin{align*}
\texttt{Env} &~=~ \texttt{Nm} \xrightharpoonup{_\textrm{fin}} \texttt{Value} & \quad
\rho &\in \texttt{Env} ~ ::= ~ \{x_1{\mapsto}v_1,\,\ldots,\,x_n{\mapsto}v_n\} \\
\texttt{Val} &~\subset~ (\texttt{Expr}\times\texttt{Env}) \;\cup\; \mathbb{Z} & \quad
v &\in \texttt{Val} ~ ::= ~ \langle \lambda x.e, \rho \rangle ~\mid~ n \\
\cdot\Longmapsto\cdot &~\subset~ (\texttt{Expr}\times\texttt{Env})\;\times\;\texttt{Val}
\\[-3em]
\end{align*}

$$
\langle x,\rho\rangle \Longmapsto \rho(x)
\qquad
\langle\lambda x.e,\rho\rangle \Longmapsto \langle\lambda x.e,\rho\rangle
$$
$$
\frac{
      \langle e_1,\rho\rangle \Longmapsto \langle\lambda x.e,\rho_1\rangle
      \quad
      \langle e_2,\rho\rangle \Longmapsto v_2
      \quad
      \langle e,\{x{\mapsto}v_2\}\rho_1\rangle \Longmapsto v
    }{\langle e_1~e_2,\rho\rangle \Longmapsto v}
$$
$$
\langle n,\rho\rangle \Longmapsto n
\qquad
\frac{\langle e_1,\rho\rangle \Longmapsto n_1 
      \quad
      \langle e_2,\rho\rangle \Longmapsto n_2}{
        \langle e_1 +\, e_2,\rho\rangle \Longmapsto n}(n\equiv n_1+n_2)
$$
$$
\frac{\langle e,\rho\rangle \Longmapsto 0
      \quad
      \langle e_0,\rho\rangle \Longmapsto n_0}{
        \langle\textbf{if}~e~\textbf{then}~e_1~\textbf{else}~e_0,\,\rho\rangle \Longmapsto n_0}
\quad        
\frac{\langle e,\rho\rangle \Longmapsto n
      \quad
      \langle e_1,\rho\rangle \Longmapsto n_1}{
        \langle\textbf{if}~e~\textbf{then}~e_1~\textbf{else}~e_0,\,\rho\rangle \Longmapsto n_1}(n\neq 0)
$$

In [1]:
:opt no-lint

In [2]:
-- 변수 이름은 문자열 나타낸다
type Nm = String

-- 람다식 문법 구조
data Expr = Var Nm             -- x
          | Lam Nm Expr        -- (\x.e)
          | App Expr Expr      -- (e1 e2)
          | Lit Int            -- n
          | Add Expr Expr      -- e1 + e2
          | If Expr Expr Expr  -- if e then e1 else e0
          deriving (Show, Eq)

In [3]:
-- 람다식을 보기좋게 문자열로 변환해주는 함수
ppExpr (Var x) = x
ppExpr (Lam x e) = "\\" ++ x ++ " -> " ++ ppExpr e
ppExpr (App e1 e2) = pp1 e1 ++ " " ++ pp2 e2
ppExpr (Lit n) = show n
ppExpr (Add e1 e2) = ppp e1 ++ " + " ++ ppp e2
ppExpr (If e e1 e0) = "if "++pp2 e++" then "++pp2 e1++" else "++pp2 e0 

pp1 e@(Lam{}) = paren (ppExpr e)
pp1 e@(Add{}) = paren (ppExpr e)
pp1 e@(If{})  = paren (ppExpr e)
pp1 e         = ppExpr e

pp2 e@(Var{}) = ppExpr e
pp2 e@(Lit{}) = ppExpr e
pp2 e         = paren (ppExpr e)

ppp e@(Var{}) = ppExpr e
ppp e@(Lit{}) = ppExpr e
ppp e@(Add{}) = ppExpr e
ppp e@(App{}) = ppExpr e
ppp e         = paren (ppExpr e)

paren s = "(" ++ s ++ ")"
brack s = "[" ++ s ++ "]"
latex s = "$" ++ s ++ "$"

-- 람다식을 보기좋게 TeX 코드로 변환해주는 함수
texExpr (Var x) = x
texExpr (Lam x e) = "\\lambda " ++ x ++ "." ++ texExpr e
texExpr (App e1 e2) = tex1 e1 ++ "~" ++ tex2 e2
texExpr (Lit n) = show n
texExpr (Add e1 e2) = texp e1 ++ "+" ++ texp e2
texExpr (If e e1 e0) = "\\texttt{if}~"++tex2 e++"~\\texttt{then}~"++tex2 e1++"~\\texttt{else}~"++tex2 e0

tex1 e@(Lam{}) = paren (texExpr e)
tex1 e@(Add{}) = paren (texExpr e)
tex1 e@(If{}) = paren (texExpr e)
tex1 t         = texExpr t

tex2 s@(Var{}) = texExpr s
tex2 s@(Lit{}) = texExpr s
tex2 s         = paren (texExpr s)

texp s@(Var{}) = texExpr s
texp s@(Lit{}) = texExpr s
texp s@(Add{}) = texExpr s
texp s@(App{}) = texExpr s
texp s         = paren (texExpr s)

In [4]:
idExpr = Lam "x" (Var "x")
ttExpr = Lam "x" (Lam "y" (Var "x")) 
ffExpr = Lam "x" (Lam "y" (Var "y")) 

putStr $ ppExpr idExpr
putStr $ ppExpr ttExpr
putStr $ ppExpr ffExpr
putStr $ ppExpr (If (Add (Lit 3) (Add (Var "x") (Var "y"))) ffExpr (App idExpr ttExpr))

\x -> x

\x -> \y -> x

\x -> \y -> y

if (3 + x + y) then (\x -> \y -> y) else ((\x -> x) (\x -> \y -> x))

In [5]:
import IHaskell.Display

html . latex $ texExpr idExpr
html . latex $ texExpr ttExpr
html . latex $ texExpr ffExpr
html . latex $ texExpr (App (App (Var "x") (Var "y")) (Var "z"))
html . latex $ texExpr (App (Var "x") (App (Var "y") (Var "z")))
html . latex $ texExpr (App (App ffExpr idExpr) ttExpr)
html . latex $ texExpr (If (Add (Lit 3) (Add (Var "x") (Var "y"))) ffExpr (App idExpr ttExpr))

$\lambda x.x$

$\lambda x.\lambda y.x$

$\lambda x.\lambda y.y$

$x~y~z$

$x~(y~z)$

$(\lambda x.\lambda y.y)~(\lambda x.x)~(\lambda x.\lambda y.x)$

$\texttt{if}~(3+x+y)~\texttt{then}~(\lambda x.\lambda y.y)~\texttt{else}~((\lambda x.x)~(\lambda x.\lambda y.x))$

In [6]:
-- call-by-value evaluator
type Env = [ (Nm, Val) ]
data Val = Clos Env Expr
         | Vint Int
         deriving Show

eval :: Env -> Expr -> Val
eval env (Var x) =
    case lookup x env of
      Nothing -> error (x ++ " not defined")
      Just v  -> v
eval env e@(Lam _ _) = Clos env e
eval env (App e1 e2) =
  case v1 of
    Clos env1 (Lam x e) -> eval ((x,v2):env1) e
    _                   -> error (show v1++" not Lam")
  where
    v1 = eval env e1
    v2 = eval env e2
eval env (Lit n) = Vint n
eval env (Add e1 e2) =
  case (v1, v2) of
    (Vint n1, Vint n2) -> Vint (n1 + n2)
    (Vint _ , _      ) -> error (show v2++" not int")
    _                  -> error (show v1++" not int")
  where
    v1 = eval env e1
    v2 = eval env e2
eval env (If e e1 e0) =
  case eval env e of
    Vint n -> if n==0 then eval env e0
                      else eval env e1
    _      -> error (show e++" not int")
eval env e =
  error (show e ++ " evaluation not defined yet")

In [7]:
import Data.List (intersperse)

texValue (Clos env e) = "\\langle"++texEnv env++","++texExpr e++"\\rangle"
texValue (Vint n) = show n

texEnv env = "\\{"
          ++ (concat . intersperse ",")
                 [x++"\\mapsto "++texValue v | (x,v) <-env]
          ++ "\\}"

In [8]:
e1 = Lam "x" $ Lam "y" $
       If (Var "x") (Add (Var "x") (Var "y"))
                    (Add (Var "y") (Var "y"))

e2 = Lit 2
e3 = Lit 3

html . latex . texValue $ eval [] e1
html . latex . texValue $ eval [] e2
html . latex . texValue $ eval [] e3

$\langle\{\},\lambda x.\lambda y.\texttt{if}~x~\texttt{then}~(x+y)~\texttt{else}~(y+y)\rangle$

$2$

$3$

In [9]:
html . latex $ texExpr (App e1 e2)
html . latex . texValue $ eval [] (App e1 e2)

$(\lambda x.\lambda y.\texttt{if}~x~\texttt{then}~(x+y)~\texttt{else}~(y+y))~2$

$\langle\{x\mapsto 2\},\lambda y.\texttt{if}~x~\texttt{then}~(x+y)~\texttt{else}~(y+y)\rangle$

In [10]:
html . latex $ texExpr (App (App e1 e2) e3)
html . latex . texValue $ eval [] (App (App e1 e2) e3)

$(\lambda x.\lambda y.\texttt{if}~x~\texttt{then}~(x+y)~\texttt{else}~(y+y))~2~3$

$5$

In [11]:
html . latex $ texExpr (App (App e1 (Lit 0)) e3)
html . latex . texValue $ eval [] (App (App e1 (Lit 0)) e3)

$(\lambda x.\lambda y.\texttt{if}~x~\texttt{then}~(x+y)~\texttt{else}~(y+y))~0~3$

$6$

---
## Recursion using a fixpoint combinator

$f(x)=x$를 만족하는 $x$를 함수 $f$의 fixpoint라고 말한다.

fixpoint combinator란 람다계산법에서 임의의 함수 $f$에 대해 $f$의 fixpoint를 찾아주는 역할을 하는 고차함수라고 생각하면 된다.

$Z$가 fixpoint combinator라는 말은 $Z(f)$가 $f$의 fixpoint 즉 $f(Z(f)) = Z(f)$라는 뜻이다.

람다계산법에서는 함수 적용에 괄호를 쓰지 않으므로 $f(Z~f) = Z~f$ 이렇게 표기하는 경우가 더 많을 것이다.

이것을 반대 방향으로 진행해보면 $Z~f = f(Z~f) = f(f(Z~f)) = f(f(f(Z~f))) = \cdots$
이렇게 함수 $f$를 몇번이고 반복 적용하는 계산을 표현할 수 있다.

In [12]:
-- Z = \f.(\x.f(\z.xxz))(\x.f(\z.xxz))
cZ = Lam "f" (App e e)
    where
        e = Lam "x" $ f `App` Lam "z" (App x x `App` z)
        f = Var "f"
        x = Var "x"
        z = Var "z"

html . latex . texExpr $ cZ

$\lambda f.(\lambda x.f~(\lambda z.x~x~z))~(\lambda x.f~(\lambda z.x~x~z))$

In [13]:
-- \f.\i.if i then i + f(i + -1) else i
e4 = Lam "f" . Lam "i" $ If i (i `Add` App f (i `Add` Lit (-1))) i          
    where
        f = Var "f"
        i = Var "i"

html . latex . texExpr $ e4

$\lambda f.\lambda i.\texttt{if}~i~\texttt{then}~(i+f~(i+-1))~\texttt{else}~i$

In [14]:
-- Z (\f.\i.if i then i + f(i + -1) else i)  100
eval [] (App cZ e4 `App` Lit 100) -- 100부터 0까지의 총합

Vint 5050

100은 너무 길어지니까 2정도로만 람다식을 손으로 전개해 보자.

```
cZ e4 2
= (\f.(\x.f(\z.xxz)) (\x.f(\z.xxz))) e4 2
= (\x.e4(\z.xxz)) (\x.e4(\z.xxz)) 2        여기서 w = (\x.e4(\z.xxz))라고 하자
= (\x.e4(\z.xxz)) w 2
= e4 (\z.w w z) 2
= (\f.\i.if i then i + f (i + -1) else i) (\z.w w z) 2
= (\i.if i then i + (\z.w w z) (i + -1) else i) 2
= if 2 then 2 + (\z.w w z) (2 + -1) else 2
= 2 + (\z.w w z) 1
= 2 + w w 1
= 2 + (\x.e4(\z.xxz)) w 1
= 2 + e4 (\z.w w z) 1
= 2 + (\f.\i.if i then i + f (i + -1) else i) (\z.w w z) 1
= 2 + (\i.if i then i + (\z.w w z) (i + -1) else i) 1
= 2 + if 1 then 1 + (\z.w w z) (1 + -1) else 1
= 2 + 1 + (\z.w w z) (1 + -1)
= 2 + 1 + w w 0
= 2 + 1 + (\x.e4(\z.xxz)) w 0
= 2 + 1 + e4 (\z.w w z) 0
= 2 + 1 + (\f.\i.if i then i + f (i + -1) else i) (\z.w w z) 0
= ...
= 2 + 1 + if 0 then 0 + f (0 + -1) else 0
= 2 + 1 + 0
```


참고로 $Y=\lambda f.(\lambda x.f(xx))(\lambda x.f(xx))$라는 $Z$보다 조금 더 간단한 fixpoint combinator가 있다.
그런데 call-by-value 또는 applicative-order 계산 방식에서는 이 $Y$-combinator는 계산을 끝내지 못하고 무한히 진행되기만 한다.
$Z$-combinator는 그래서 call-by-value 또는 applicative-order 에서도 재귀적 계산을 표현하는 데 활용할 수 있다.
$Y$나 $Z$ 말고도 람다계산법에서 fixpoint-combinator들이 더 존재한다.